In [1]:
import pandas as pd
import numpy as np

In [2]:
iowa = 'data/iowa_caucus_results.xlsx'
new_hampshire = 'data/new_hampshire_caucus_results.xlsx'
nevada = 'data/nevada_caucus_results.xlsx'
south_carolina = 'data/south_carolina_caucus_results.xlsx'


In [3]:
iowa = pd.read_excel(iowa)
new_hampshire = pd.read_excel(new_hampshire)
nevada = pd.read_excel(nevada)
south_carolina = pd.read_excel(south_carolina)

In [77]:
active_candidates = ['Bernie Sanders',
                       'Joe Biden', 
                       'Elizabeth Warren', 
                       'Amy Klobuchar', 
                       'Tom Steyer', 
                       'Pete Butigieg', 
                       'Michael Bloomber']
active_candidates_LN = ['Sanders',
                       'Biden', 
                       'Warren', 
                       'Klobuchar', 
                       'Steyer', 
                       'Butigieg', 
                       'Bloomber']

In [119]:
#### This Code works for Data from Wikipedia

new_hampshire_df = new_hampshire.drop('State-wide', axis=1)
new_hampshire_df = new_hampshire_df.melt(id_vars=['Candidate'],value_vars=new_hampshire_df.columns.to_list()[1:-1])
new_hampshire_df = new_hampshire_df[~new_hampshire_df['value'].isin(['—'])]
new_hampshire_df['value'] = new_hampshire_df['value'].astype(int)
new_hampshire_df['State'] = "NH"
new_hampshire_df = new_hampshire_df[new_hampshire_df['Candidate'] != "Total"]
new_hampshire_df = new_hampshire_df[new_hampshire_df['Candidate'].isin(active_candidates)]
new_hampshire_df = new_hampshire_df.sort_values(by=['variable', 'value'], ascending=False)
new_hampshire_counties = new_hampshire_df['variable'].unique()

for county in new_hampshire_counties:
    i = 0
    for index, row in new_hampshire_df.iterrows():
        if row['variable'] == county:
            i = i + 1
            if i == 1:
                new_hampshire_df.loc[index, 'winner'] = 1
            else:
                new_hampshire_df.loc[index, 'winner'] = 0

        candidate = row["Candidate"].split()
        new_hampshire_df.loc[index, 'last name'] = candidate[1]
new_hampshire_df.rename(columns={"variable": "county", "value": "votes", "State": "state"}, inplace=True)
new_hampshire_df.drop("Candidate", axis=1, inplace=True)
new_hampshire_df = new_hampshire_df[['last name', 'county', 'votes', 'state', 'winner']]

In [113]:
### This code works for data from Washington Post
iowa_df = iowa.transpose().reset_index()
iowa_df.columns = iowa_df.iloc[0]
iowa_df = iowa_df.drop(iowa_df.index[0])
iowa_df = iowa_df.melt(id_vars=['Counties'],value_vars=iowa_df.columns.to_list()[1:-1])
iowa_df['State'] = "IA"
iowa_df = iowa_df[iowa_df['Counties'].isin(active_candidates_LN)]
iowa_df = iowa_df.sort_values(by=[0, 'value'], ascending=False)
iowa_counties = iowa_df[0].unique()

for county in iowa_counties:
    i = 0
    for index, row in iowa_df.iterrows():
        if row[0] == county:
            i = i + 1
            if i == 1:
                iowa_df.loc[index, 'winner'] = 1
            else:
                iowa_df.loc[index, 'winner'] = 0
iowa_df.rename(columns={"Counties": "last name", 0: "county", "State": "state", 'value': 'votes'}, inplace=True)    

In [125]:
results = new_hampshire_df.append(iowa_df)
results = results.append(nevada_df)
results = results.append(south_carolina_df)

In [128]:
results.to_csv('results.csv')